## Introduction

The titanic machine learning competition....

# 1. Import libraries



In [ ]:
import pandas as pd
import numpy as np
import missingno
from collections import Counter

# Data visualisation
import seaborn as sns
import matplotlib.pyplot as plt

# Data preprocessing
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

# Machine learning models
from sklearn.linear_model import LogisticRegression, Perceptron, SGDClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from numpy import array
import matplotlib.pyplot as pl
from sklearn.ensemble import VotingClassifier

# Model evaluation
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import learning_curve
# Tuning
from sklearn.feature_selection import RFECV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV


# Remove warnings
import warnings
warnings.filterwarnings('ignore')

# 2. Import and read data



In [ ]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [ ]:
train.head()

In [ ]:
test.head()

# 3. Data description


The data provided is split into two groups:
1) The training set (train.csv)
2) The testing set (test.csv)

The training set includes a survival column which indicates whether or not the passenger survived. This data set is used to create the machine learning model.
The testing set is used to determine how well the model (generated from the training data set) performs on new unseen data. The testing data set does not provide the passengers' survival status. The model generated predicts the passengers' survival status.

The table below provides all the relevant information about the columns in the data sets:

| Column Name          | Description                                                | Key                    |
| ---------------------| ---------------------------------------------------------- | ---------------------- |
| __PassengerId__      | Passenger Identity                                         |                        | 
| __Survived__         | Passenger survival status                                  | 0 = No, 1 = Yes        | 
| __Pclass__           | Ticket class, a representation of socio-economic status (SES)| 1 = 1st class, 2 = 2nd class, 3 = 3rd class | 
| __Name__             | Passenger's name                                           |                        | 
| __Sex__              | Passenger's sex                                            |                        |
| __Age__              | Passengers age (in years)                                  |                        |
| __SibSp__            | Number of sibling and/or spouse travelling with passenger  |                        |
| __Parch__            | Number of parent and/or children travelling with passenger |                        |
| __Ticket__           | Ticket number                                              |                        |
| __Fare__             | Price of the ticket                                        |                        |
| __Cabin__            | Cabin number                                               |                        |
| __Embarked__         | Point of embarkation                                       | C = Cherbourg, Q = Queenstown, S = Southampton |

More information can be found under the [data](https://www.kaggle.com/c/titanic/)

# 4. Data Analysis 


Exploratory data analysis is used to gain insight on the data provided. This is achieved by using visualisation tools such as graphs and tables. It will allow us to understand the data and derive preliminary conclusions. Furthermore, it will summerise important trends, characteristics, and abnormalities in the dataset which will ultimately aid in training the model.

The following is explored and analysed:
- Data Types
- The shape of the data
- Missing values in the data
- Statistics derived from the data

## 4.1 Data types,data shapes, missing data and summary statistics

### 4.1.1.Data Types

In [ ]:
# Non-null count and data types of the training
train.info()

 <font color='pink'>Observation:</font>  The training-set has 891 rows and 11 features including the __target variable (survived).__ 2 of the features are floats, 5 are integers and 5 are objects. When training, the model requires the data to all be in the form of numbers, therefore these columns will be converted later on.

### 4.1.2.Data Shape

In [ ]:

print("The shape of the training data set: ", train.shape)
print("The shape of the testing data set: ", test.shape)

<font color='pink'>Observation:</font> The testing data set has one column less column than the training data set (the Survived column). As discussed above in section 3, survived is our response/target variable and will therefore be determined from the model derived from the training data.

### 4.1.3.Missing Values

In [ ]:

#Determine what percentage of data is missing values in each column of the training dataset
totalNumberOfDataPoints = train.isnull().sum().sort_values(ascending=False)
percentMissing = train.isnull().sum()/train.isnull().count()*100
percentMissingRounded = (round(percentMissing, 1)).sort_values(ascending=False)
missingData = pd.concat([totalNumberOfDataPoints, percentMissingRounded], axis=1, keys=['Total missing', '%'])
missingData.head(13)

In [ ]:

#Determine what percentage of data is missing values in each column of the testing dataset
totalNumberOfDataPoints = test.isnull().sum().sort_values(ascending=False)
percentMissing = test.isnull().sum()/test.isnull().count()*100
percentMissingRounded = (round(percentMissing, 1)).sort_values(ascending=False)
missingData = pd.concat([totalNumberOfDataPoints, percentMissingRounded], axis=1, keys=['Total missing', '%'])
missingData.head(13)

<font color='pink'>Observation:</font> From the two tables above it can be seen that the training set has missing values in the Cabin, Age and Embarked columns. The testing dataset has missing values in the Cabin, Age and Fare columns. 
For the training dataset, the Embarked column only contains two missing values which can be easily dropped or filled. The Age column on the other hand has 177 missing values. We therefore, cannot drop the rows which have missing values in the age column as this will eliminate 20% of the training data. Therefore, these values need to be filled in. The approach taken to fill in the missing values is discussed below in section 5.2. Since the Cabin column is missing 77% of data points, we have decided to drop this column. 

### 4.1.4.Statistics

In [ ]:

# Summary of the statistics for the training data set 
train.describe()


The table above gives an overview of the central tendencies of the numeric data in the testing dataset. <br /> <font color='pink'>Observations:</font> 
- 38% of people in the training dataset survived the Titanic 
- The passenger age ranges from 0.4 to 80 years old.
- There is an outlier in the Fare column because of the differences between the 75th percentile, standard deviation, and the max value (512). We will thus determine how to deal with this outlier by either dropping its corresponding row or filling the outlier with an appropriate value. 

## 4.2 Feature analysis


For feature analysis the training dataset will be split into two categories:
1) Categorical variables
2) Numerical variables

Categorical variables have values belonging to one of two or more categories. Numerical variables have a continuous distribution.
Identifying which variables are categorical and which variables are numerical will hel structure the data analysis properly. For example it makes no sense to determine the average of a categorical variable such as sex or class. Furthermore, sex, class and embarked have no intrinsic ordering to its value. 

### 4.2.1 Categorical variables


In this data set the categorical variables are:
1) Sex
2) Pclass 
3) Embarked.

#### 4.2.1.1.Categorical variable: Sex

In [ ]:
# Value counts of the sex column
train['Sex'].value_counts(dropna = False)

<font color='pink'>Observation:</font> There are 263 more male passengers than female passengers in the training dataset. Therefore there it is assumend that the test dataset will have a similiar distribution of sex.

In [ ]:

# Mean of survival according to sex
train[['Sex', 'Survived']].groupby('Sex', as_index = False).mean().sort_values(by = 'Survived', ascending = False)

In [ ]:

# visualisation for the probability of survival according to sex
sns.barplot(x = 'Sex', y ='Survived', data = train)
plt.ylabel('Probability of survival')
plt.title('Survival Probability by Sex')

<font color='pink'>Observation:</font> Female passengers are more likely to survive.

#### 4.2.1.2.Categorical variable: Pclass

In [ ]:

# Value counts of the Pclass column in the training dataset

train['Pclass'].value_counts(dropna = False)

In [ ]:

# Mean of survival by passenger class in the training dataset

train[['Pclass', 'Survived']].groupby(['Pclass'], as_index = False).mean().sort_values(by = 'Survived', ascending = False)

In [ ]:
 
#Pclass distributions for survived and not survived
ax=sns.kdeplot(train.loc[(train['Survived'] == 0),'Pclass'],shade=True,color='r',label='Not Survived')
ax.legend()
ax=sns.kdeplot(train.loc[(train['Survived'] == 1),'Pclass'],shade=True,color='b',label='Survived')
ax.legend()

plt.title("Passenger Class Distribution - Survived vs Non-Survived", fontsize = 25)
labels = ['First', 'Second', 'Third']
plt.xticks(sorted(train.Pclass.unique()),labels);

In [ ]:

sns.barplot(x = 'Pclass', y ='Survived', data = train)
plt.ylabel('Survival Probability')
plt.title('Survival Probability by Passenger Class')

<font color='pink'>Observation:</font> The probability of survival decreases with a decrease in passenger class. It can therefore be assumed that first class passengers were prioritised during the evacuation. Evidently, from the two graphs above, Pclass plays an important role in determining whether a passenger did or did not survive. According to the training dataset, 63% of the 1st class passengers survived, 48% of the 2nd class passengers survived and only 24% of the 3rd class passengers survived.

#### 4.2.1.3.Categorical variables combined: Sex and Plass

In [ ]:

# Survival by gender and passenger class
sns.factorplot(x = 'Pclass', y = 'Survived', hue = 'Sex', data = train, kind = 'bar').despine(left = True)
plt.ylabel('Survival Probability')
plt.title('Survival Probability by Sex and Passenger Class')

<font color='pink'>Observation:</font>The graph above indicates that in every class, females where always more likely to survive. It can also be seen that males in the first class were more likely to survive than in any other class.

#### 4.2.1.4.Categorical variable: Embarked

In [ ]:
# Value counts of the Embarked column 
#NAN is the missing values in Embarked
train['Embarked'].value_counts(dropna = False)

In [ ]:
# Mean of survival by point of embarkation
train[['Embarked', 'Survived']].groupby(['Embarked'], as_index = False).mean().sort_values(by = 'Survived', ascending = False)

In [ ]:
#Visualisation for the probability of survival according to point of embarkation
sns.barplot(x = 'Embarked', y ='Survived', data = train)
plt.ylabel('Probability of Survival')
plt.title('Survival Probability by Point of Embarkation')

<font color='pink'>Observation:</font> The probability of survival is highest for location C and lowest for location S.
Perhaps first class passengers embarked from location C and therefore because first class passengers had a higher chance of survival, location c also has the highest chance of survival. As an alternative perhaps third class passengers embarked from location S and because third class passengers had the lowest chance of survival , location S also has the lowest survival probability. This hypothesis is tested in section 4.2.1.5 below. 

#### 4.2.1.5.Categorical variable combined: Embarked and Class

In [ ]:
#Gia
# Visualisation for the relationship between class and embark 
sns.factorplot('Pclass', col = 'Embarked', data = train, kind = 'count')

<font color='pink'>Observation:</font> The hypothesis discussed in section 4.1.2.4 appears to be correct.  Location S has majority of the third class passengers and the majority of passengers embarking from location C are first class.

### 4.2.2 Numerical variables

Gia
In this dataset, the numerical variables are:
1) SibSp
2) Parch
3) Age
4) Fare

#### 4.2.2.1.Numerical variables correlation with survival

In [ ]:
#Heatmap of numerical variables
df_num = train[['Survived','Age','SibSp','Parch','Fare']]
sns.heatmap(df_num.corr(), annot=True,cmap="RdBu")
plt.title("Correlations Among Numeric Features", fontsize = 18);

<font color='pink'>Observation:</font> The heatmap displayed above shows that Parch and SiSp often travel together.Therefore it will useful to create a isAlone and a family size feature.
we also see that Fare has a pretty large positive correlation to survival, thus it may be an important metric in training the model. 

#### 4.2.2.2.Numerical variable: SibSp

In [ ]:
# Value counts of the SibSp column 
train['SibSp'].value_counts(dropna = False)

In [ ]:
# Mean of survival by SibSp
train[['SibSp', 'Survived']].groupby('SibSp', as_index = False).mean().sort_values(by = 'Survived', ascending = False)

In [ ]:
#Visualisation for probability of survival according to SiSP 
sns.barplot(x = 'SibSp', y ='Survived', data = train)
plt.ylabel('Survival Probability')
plt.title('Survival Probability by SibSp')

#### 4.2.2.3.Numerical variable: Parch

In [ ]:
# Value counts of the Parch column 
train['Parch'].value_counts(dropna = False)

In [ ]:
# Mean of survival by Parch
train[['Parch', 'Survived']].groupby('Parch', as_index = False).mean().sort_values(by = 'Survived', ascending = False)

In [ ]:
#Visualisation for probability of survival according to Parch
sns.barplot(x = 'Parch', y ='Survived', data = train)
plt.ylabel('Survival Probability')
plt.title('Survival Probability by Parch')

#### 4.2.2.4.Numerical variable: Age

In [ ]:
# Passenger age distribution
sns.distplot(train['Age'], label = 'Skewness: %.2f'%(train['Age'].skew()))
plt.legend(loc = 'best')
plt.title('Passenger Age Distribution')

In [ ]:
# Age distribution by survival
sns.FacetGrid(train, col = 'Survived').map(sns.distplot, 'Age')

In [ ]:
sns.kdeplot(train['Age'][train['Survived'] == 0], label = 'Did not survive')
sns.kdeplot(train['Age'][train['Survived'] == 1], label = 'Survived')
plt.xlabel('Age')
plt.legend()
plt.title('Passenger Age Distribution by Survival')

#### 4.2.2.5.Numerical variable: Fare

In [ ]:
# Passenger fare distribution
sns.distplot(train['Fare'], label = 'Skewness: %.2f'%(train['Fare'].skew()))
plt.legend(loc = 'best')
plt.ylabel('Passenger Fare Distribution')

<font color='pink'>Observation:</font> The majority of passengers paid low fares where there are few people who paid very large fares.

#### 4.2.2.6.Detect outliers in numerical variables

Outliers are points in the dataset that don't conform with majority of the data (they are extreme values). Outliers need to be addressed as they tend to skew data and can cause inaccurate model predictions. The Tukey method is used to detect these outliers. Outliers can only be determined for numeric data.

In [ ]:
#Function to predict outliers
def detect_outliers(df, n, features):
    """"
    This function loops through the list of features and detects outliers in each feature. A data point is considered to be 
    an outlier if it is less than Q1-1.5*IQR or if it is greater than Q3+1.5*IQR. Once the outliers have been determined for 
    a feature, their indices will be stored in a list and then the loop will proceed to the next feature. This process repeats
    until the last feature is complete. Finally, using the list with the indices of the outliers, the frequency of outliers is
    determined and if the frequency is greater than n then the list fill be returned.    
    """
    outlierIndices = [] 
    for col in features: 
        Q1 = np.percentile(df[col], 25)
        Q3 = np.percentile(df[col], 75)
        IQR = Q3 - Q1
        outlierStep = 1.5 * IQR 
        outlierList = df[(df[col] < Q1 - outlierStep) | (df[col] > Q3 + outlierStep)].index
        outlierIndices.extend(outlierList) 
    outlierIndices = Counter(outlierIndices)
    multipleOutliers = list(key for key, value in outlierIndices.items() if value > n) 
    return multipleOutliers
outliers_to_drop_test = detect_outliers(test, 2, ['Age', 'SibSp', 'Parch', 'Fare'])
outliers_to_drop = detect_outliers(train, 2, ['Age', 'SibSp', 'Parch', 'Fare'])
print("The indices where outliers occur are {}: ".format(len(outliers_to_drop)), outliers_to_drop) 

In [ ]:
# Outliers in numerical variables
#Visualise the 10 rows identified above as rows containing outliers
train.loc[outliers_to_drop, :]

### 4.2.3 Correlation between categorical and numerical

#### 4.2.3.1.All variables

In [ ]:
sns.heatmap(train[['Survived', 'SibSp', 'Parch', 'Age', 'Fare','Pclass']].corr(), annot = True, fmt = '.2f', cmap='RdBu')

<font color='pink'>Observation:</font>  Fare appears to have a high correlation with survival and Pclass has a high ngeative correlation with survival.

# 5. Data preprocessing

Now that we know what features are correlated and some other important factors, we can get the dataset into a form to be modelled and trained. This includes:
- Dealing with ouliers
- Drop and fill missing values
- Data transformation 
- Feature engineering
- Feature encoding

## 5.1 Remove Outliers

In [ ]:
# Drop outliers 

print("Train Set Before: {} rows".format(len(train)))
#train = train.drop(outliers_to_drop, axis = 0).reset_index(drop = True)
print("Train Set After: {} rows".format(len(train)))
print("Test Set Before: {} rows".format(len(test)))
# test = test.drop(outliers_to_drop_test, axis = 0).reset_index(drop = True)
print("Test Set After: {} rows".format(len(test)))

<font color='pink'>Observation:</font>  Dropping outliers does not seem to improve scores later on, therefore only drop for specific Machine Learning Algorithms. 

## 5.2 Drop and fill missing values

In [ ]:
# Drop ticket and cabin features from training and test set as they are unique or missing many values
train = train.drop(['Ticket', 'Cabin'], axis = 1)
test = test.drop(['Ticket', 'Cabin'], axis = 1)

In [ ]:
train.isnull().sum().sort_values(ascending = False)

In [ ]:
# Fill missing value in Embarked with mode as only 3 values
mode = train['Embarked'].dropna().mode()[0]
train['Embarked'].fillna(mode, inplace = True)

In [ ]:
test.isnull().sum().sort_values(ascending = False)

In [ ]:
# Fill missing value for Fare 
median = test['Fare'].dropna().median()
test['Fare'].fillna(median, inplace = True)

In [ ]:
# Check where indeces of missing ages are
age_nan_indices_train = list(train[train['Age'].isnull()].index)
len(age_nan_indices_train)
age_nan_indices_test = list(test[test['Age'].isnull()].index)


Age is negatively correlated with SibSp, Parch and Pclass as shown in section 4. Loop through each of the rows which have the same corresponding values and fill the missing age with their median. Otherwise fill with the Age median.

In [ ]:
median_age = train['Age'].median()

for index in age_nan_indices_train:
    predict_age = train['Age'][(train['SibSp'] == train.iloc[index]['SibSp']) 
                                 & (train['Parch'] == train.iloc[index]['Parch'])
                                 & (train['Pclass'] == train.iloc[index]["Pclass"])].median()
    if np.isnan(predict_age):
        train['Age'].iloc[index] = median_age
    else:
        train['Age'].iloc[index] = predict_age
combine = pd.concat([train, test], axis = 0).reset_index(drop = True)
median_age = combine['Age'].median()
for index in age_nan_indices_test:
    #use larger sample to fill test data 
    test['Age'].iloc[index] = median_age  

In [ ]:
# Make sure there are no more missing ages 
print(train['Age'].isnull().sum())
test['Age'].isnull().sum()

## 5.3 Data transformation

Apply a log transformation to fair as it has a high right-skewnewss

In [ ]:
#  fare distribution

sns.distplot(train['Fare'], label = 'Skewness: %.2f'%(train['Fare'].skew()))
plt.legend(loc = 'best')
plt.title('Passenger Fare Distribution')

In [ ]:
# In order to reduce skewness in fare, apply log transformation 
train['Fare'] = train['Fare'].map(lambda x: np.log(x) if x > 0 else 0)
test['Fare'] = test['Fare'].map(lambda x: np.log(x) if x > 0 else 0)

In [ ]:
# After log transformation

sns.distplot(train['Fare'], label = 'Skewness: %.2f'%(train['Fare'].skew()))
plt.legend(loc = 'best')
plt.title('Fare Distribution After Log Transformation')

## 5.4 Feature engineering

We create new features from existing features to obtain an improved model.

### 5.4.1 Title

In [ ]:
train.head()

In [ ]:
#Title from name column
train['Title'] = [name.split(',')[1].split('.')[0].strip() for name in train['Name']]
train[['Name', 'Title']].head()
test['Title'] = [name.split(',')[1].split('.')[0].strip() for name in test['Name']]
test[['Name', 'Title']].head()

In [ ]:
# Value counts of Title
train['Title'].value_counts()

In [ ]:
# visualise the testing titles
test['Title'].value_counts()

Simplify Title as there are several unique titles that do not necessarily have a trend

In [ ]:
train['Title'] = train['Title'].replace(['Dr', 'Rev', 'Col', 'Major', 'Lady', 'Jonkheer', 'Don', 'Capt', 'the Countess',
                                             'Sir'], 'Rare')
train['Title'] = train['Title'].replace(['Mlle', 'Ms'], 'Miss')
train['Title'] = train['Title'].replace('Mme', 'Mrs')

test['Title'] = test['Title'].replace(['Dr', 'Rev', 'Col',  'Capt', 'Dona'], 'Rare')
test['Title'] = test['Title'].replace(['Ms'], 'Miss')


Drop the name column as title has been extracted

In [ ]:
train = train.drop('Name', axis = 1)
train.head()

test = test.drop('Name', axis = 1)
test.head()

In [ ]:
train[['Title', 'Survived']].groupby(['Title'], as_index = False).mean().sort_values(by = 'Survived', ascending = False)

<font color='pink'>Observation:</font> Woman and young males had a high chance of survival.

### 5.4.2 byThemselves

Simplify and summarize data to show if passenegers were alone, as opposed to information about both siblings and parents

In [ ]:
train['FamilySize'] = train['SibSp'] + train['Parch'] + 1
train[['SibSp', 'Parch', 'FamilySize']].head()

test['FamilySize'] = test['SibSp'] + test['Parch'] + 1
test[['SibSp', 'Parch', 'FamilySize']].head()

In [ ]:
# Create byThemselves feature as familySize may have more information than we need, leading to overfitting

train['byThemselves'] = 0
train.loc[train['FamilySize'] == 1, 'byThemselves'] = 1

test['byThemselves'] = 0
test.loc[test['FamilySize'] == 1, 'byThemselves'] = 1

In [ ]:
# Drop SibSp, Parch and FamilySize as this is contained in byThemselves

train = train.drop(['SibSp', 'Parch','FamilySize'], axis = 1)
test = test.drop(['SibSp', 'Parch','FamilySize'], axis = 1)
train.head()

### 5.4.3 Age*Class

First convert Age into an ordinal variable. Group Ages into 4 age bands 

In [ ]:
train['AgeBand'] = pd.cut(train['Age'], 4)
test['AgeBand'] = pd.cut(test['Age'], 4)
print(train['AgeBand'].head(10))

In [ ]:
train.loc[train['Age'] <= 20.315, 'Age'] = 0
train.loc[(train['Age'] > 20.315) & (train['Age'] <= 40.21), 'Age'] = 1
train.loc[(train['Age'] > 40.21) & (train['Age'] <= 60.105), 'Age'] = 2
train.loc[train['Age'] > 60.105,'Age'] = 3

test.loc[test['Age'] <= 20.315, 'Age'] = 0
test.loc[(test['Age'] > 20.315) & (test['Age'] <= 40.21), 'Age'] = 1
test.loc[(test['Age'] > 40.21) & (test['Age'] <= 60.105), 'Age'] = 2
test.loc[test['Age'] > 60.105,'Age'] = 3

# Drop age band feature
train = train.drop('AgeBand', axis = 1)
test = test.drop('AgeBand', axis = 1)

In [ ]:
# Convert ordinal Age into integer
train['Age'] = train['Age'].astype('int')
test['Age'] = test['Age'].astype('int')
train['Age'].dtype

In [ ]:
# Create Age*Class

train['Age*Class'] = train['Age'] * train['Pclass']
test['Age*Class'] = test['Age'] * test['Pclass']
train[['Age', 'Pclass', 'Age*Class']].head()

In [ ]:
# Bin Fare 
train['FareBand'] = pd.qcut(train['Fare'], 4)
test['FareBand'] = pd.qcut(test['Fare'], 4)
train['FareBand'].head(10)


In [ ]:
#ordinal encoding, simliar to age
train.loc[train['Fare'] <= 2.066, 'Fare'] = 0
train.loc[(train['Fare'] > 2.066) & (train['Fare'] <= 2.671), 'Fare'] = 1
train.loc[(train['Fare'] > 2.671) & (train['Fare'] <= 3.418), 'Fare'] = 2
train.loc[train['Fare'] > 3.418, 'Fare'] = 3

test.loc[test['Fare'] <= 2.066, 'Fare'] = 0
test.loc[(test['Fare'] > 2.066) & (test['Fare'] <= 2.671), 'Fare'] = 1
test.loc[(test['Fare'] > 2.671) & (test['Fare'] <= 3.418), 'Fare'] = 2
test.loc[test['Fare'] > 3.418, 'Fare'] = 3

In [ ]:
train = train.drop(['FareBand'], axis = 1)
test = test.drop(['FareBand'], axis = 1)

In [ ]:
# Convert Fare into integer
train['Fare'] = train['Fare'].astype('int')
test['Fare'] = test['Fare'].astype('int')

## 5.5 Feature encoding 

Variables must be numeric to use for machine learning. Age and Fare were done when Binning. 

In [ ]:
train.head()

In [ ]:
label = LabelEncoder() 
train['Embarked'] = label.fit_transform(train['Embarked'])
test['Embarked'] = label.fit_transform(test['Embarked'])
train['Title'] = label.fit_transform(train['Title'])
test['Title'] = label.fit_transform(test['Title'])
train['Sex'] = train['Sex'].map({'male': 0, 'female': 1})
test['Sex'] = test['Sex'].map({'male': 0, 'female': 1})

train.head()

In [ ]:
train = train.drop('PassengerId', axis = 1)
train.head()

In [ ]:
train['Survived'] = train['Survived'].astype('int')
train.head()

In [ ]:
test.head()

In [ ]:
sns.heatmap(train[['Survived', 'Pclass', 'Sex', 'Age', 'Fare','Embarked','Title','byThemselves','Age*Class']].corr(), annot = True, fmt = '.2f', cmap='RdBu')

After data preprocessing, we can see how all of the new and remaining features are correlated in the figure above. Clearly sex has the greatest correlation with survived.

# 6. Modelling

For the modelling part of this project we will utilise the Scikit-learn library for machine learning. 

 As discussed above, this is a classfication problem, so consequently will use classfication models for our training.  We have chosen to investigate the performance of the following classifiers as these are commonly used classifiers:

- Logistic regression
- Multi Layer Perceptron
- K-nearest neighbours
- Gaussian naive bayes
- Linear SVC
- Stochastic gradient descent
- Random forest
- Support vector machines

In this section , we will fit the models to the training data set and evaluate the models' prediction accuracy. Further on we  will implement feature tuning and hyperparameter tuning to further boost the performance of the the following models:

- Logistic regression
- Multi Layer Perceptron
- K-nearest neighbours
- Random Forest 



In [ ]:
X_train = train.drop('Survived', axis = 1)
Y_train = train['Survived']
X_test = test.drop('PassengerId', axis = 1).copy()

## 6.1 K-fold cross validation

It is important to not get too carried away with models with impressive training accuracy as what we should focus on instead is the model's ability to predict out-of-samples data, in other words, data our model has not seen before.

This is where k-fold cross validation comes in. K-fold cross validation is a technique whereby a subset of our training set is kept aside and will act as holdout set for testing purposes. Here is a great [video](https://www.youtube.com/watch?v=fSytzGwwBVw) explaining the concept in more detail. 

We compute the accuracy and cross-entropy log-loss for each algorithm to determine its performance metrics. 

In [ ]:
# Create a list which contains classifiers 
classifiers = [ MLPClassifier(),
                LogisticRegression(), 
                GaussianNB(),
                KNeighborsClassifier(n_neighbors = 5), 
                SVC(),  
                SGDClassifier(), 
                LinearSVC(),
                RandomForestClassifier(),
               ]

if len(classifiers) is not 8: 
    print("error")

cross_val_results = []
cross_val_losses = []

for classifier in classifiers:
    cross_val_results.append(cross_val_score(classifier, X_train, Y_train, scoring = 'accuracy', cv = 5))
    cross_val_losses.append(cross_val_score(classifier, X_train, Y_train, scoring = 'neg_log_loss', cv = 5))

In [ ]:
# Mean and standard deviation of cross validation results for each classifier  

cross_val_mean = []
cross_val_std = []
cross_val_loss_mean =[]
cross_val_loss_std = []

for cross_val_result in cross_val_results:
    cross_val_mean.append(cross_val_result.mean())
    cross_val_std.append(cross_val_result.std())

for cross_val_loss in cross_val_losses:
    cross_val_loss_mean.append(cross_val_loss.mean())
    cross_val_loss_std.append(cross_val_loss.std())    

    
    
#putting our cross val mean and standard deviation for each algorithm into a data frame
cross_val_res = pd.DataFrame({'Cross Validation Mean': cross_val_mean, 'Cross Validation Std': cross_val_std, 'Algorithm': [ 'Multilayer perceptron','Logistic Regression','Naive Bayes', 'K Nearest Neighbours', 'Support Vector Machines', 'Stochastic Gradient Decent', 'Linear SVC', 'Random Forest']})
cross_val_los = pd.DataFrame({'Cross Validation Mean': cross_val_loss_mean, 'Cross Validation Std': cross_val_loss_std, 'Algorithm': [ 'Multilayer perceptron','Logistic Regression','Naive Bayes', 'K Nearest Neighbours', 'Support Vector Machines', 'Stochastic Gradient Decent', 'Linear SVC', 'Random Forest']})
print('Table Showing Cross Validation Mean Accuracy')
print(cross_val_res.sort_values(by = 'Cross Validation Mean', ascending = False, ignore_index = True))
print('\nTable Showing Cross Validation Mean Loss')
print(cross_val_los.sort_values(by = 'Cross Validation Mean', ascending = True, ignore_index = True))

<font color='pink'>Observation:</font> Support Vector Machines has the greatest accuracy followed by Random Forest and MLP. However, MLP has the least observable loss and would appear to be the 'best' at first glance. 

Nonetheless, 4 algorithms will be tuned below and the scores observed once more. 

In [ ]:
sns.barplot('Cross Validation Mean', 'Algorithm', data = cross_val_res, order = cross_val_res.sort_values(by = 'Cross Validation Mean', ascending = False)['Algorithm'], palette = 'Set3', **{'xerr': cross_val_std})
plt.ylabel('Algorithm')
plt.title('Cross Validation Scores')

In [ ]:
sns.barplot('Cross Validation Mean', 'Algorithm', data = cross_val_los, order = cross_val_los.sort_values(by = 'Cross Validation Mean', ascending = False)['Algorithm'], palette = 'Set3', **{'xerr': cross_val_loss_std})
plt.ylabel('Algorithm')
plt.title('Cross Validation Losses')

## 6.2 Tuning

### 6.2.1 Feature Tuning

The best remaining features can be found for each algorithm and the rest can be dropped for each respective algorithm. 

Logistic Regression

In [ ]:
# the number of correct classifications (survived or did not survive) is proportional to the "accuracy" scoring
RFECV1 = RFECV(estimator=LogisticRegression(), step=1, cv=10, scoring='accuracy')
RFECV1.fit(X_train, Y_train)

print("Optimal number of features: %d" % RFECV1.n_features_)
print('Selected features: %s' % list(X_train.columns[RFECV1.support_]))

# Plot the number of features VS. the CV scores
plt.xlabel("Number of features that are selected")
plt.ylabel("Cross validation score ")
plt.plot(range(1, len(RFECV1.grid_scores_) + 1), RFECV1.grid_scores_)
plt.show()

In [ ]:
X_train_LR = X_train.drop(['byThemselves', 'Fare'], axis =1).copy()
X_test_LR = X_test.drop(['byThemselves', 'Fare'], axis =1).copy()

Random Forest

In [ ]:
RFECV2 = RFECV(estimator=RandomForestClassifier(), step=1, cv=10, scoring='accuracy')
RFECV2.fit(X_train, Y_train)

print("Optimal number of features: %d" % RFECV2.n_features_)
print('Selected features: %s' % list(X_train.columns[RFECV2.support_]))

# Plot the number of features VS. the CV scores
plt.xlabel("Number of features that are selected")
plt.ylabel("Cross validation score ")
plt.plot(range(1, len(RFECV2.grid_scores_) + 1), RFECV2.grid_scores_)
plt.show()


In [ ]:
#based on the above, for Random Forest we should drop 3 columns
X_train_RF = X_train.drop(['Age'], axis =1).copy()
X_test_RF = X_test.drop(['Age'], axis =1).copy()

MLP and KNN have intrinsic feature importance selection and so do not need to have it manually performed. 

### 6.2.2 Hyperparameter Tuning
Hyperparameter tuning is the process of tuning the parameters of a model.

#### 6.2.2.1 Hyperparameter tuning for MLP

In [ ]:
mlp = MLPClassifier(max_iter=50)
param_grid = {'alpha': [1,1e-3,1e-5],  
              'hidden_layer_sizes': [(50,50,50),(3,), (50,190,3), (64,64,64) ],
              'activation': ['tanh','relu','logistic'],  
              'solver': ['sgd','adam'],
              'learning_rate': ['constant','adaptive']
}
grid = GridSearchCV(mlp, param_grid,n_jobs=-1, cv=5) 
grid.fit(X_train, Y_train)
MLP_tuned= grid.best_estimator_
MLP_trained_tuned=MLP_tuned.fit(X_train, Y_train)
print("Best parameters: ", grid.best_params_) 



The multilayer perceptron(MLP) is an Artificial neural network with one or more  hidden layers. The MLP has the advantage over the Perceptron algorithm in that it is able to handle non linear mapping of inputs to outputs. The MLP is a feedforward algorithm that performs a weighted summation of the inputs and their respective weights. This summation is then fed into a activation function such as ReLU to determine the output. The output of each of this nodes is then fed into the next hidden layers and then to the output. The MLP also has back propagation which allows it to iteratively adjust the weights in the network to reduce the cost function. The MLP was chosen for hyperparameter tuning as it has one of the highest accuracy scores and one of the lowest standard deviations (Bento, 2021).


Using the GridSearchCV method with the following parameters: a grid of hyperparameters and a K-fold cross validation value. The parameter grid includes the alpha value used for regularization(which reduces overfitting), values for how many nodes in each of the 3 hidden layers , the activation function , the solver used for backpropagation and the learning rate. The gridSearchCV then finds the parameters which result in the highest accurcay (1.17. Neural network models (supervised), n.d.).

Bento, C., 2021. Multilayer Perceptron Explained with a Real-Life Example and Python Code: Sentiment Analysis. [online] Medium. Available at: <https://towardsdatascience.com/multilayer-perceptron-explained-with-a-real-life-example-and-python-code-sentiment-analysis-cb408ee93141> [Accessed 14 June 2022].

scikit-learn. n.d. 1.17. Neural network models (supervised). [online] Available at: <https://scikit-learn.org/stable/modules/neural_networks_supervised.html> [Accessed 14 June 2022].

(Aiden)

#### 6.2.2.2 Hyperparameter tuning for KNN



In [ ]:
leaf_size = list(range(1,50))
n_neighbors = list(range(1,30))
p=[1,2]
hyperparameters = dict(leaf_size=leaf_size, n_neighbors=n_neighbors, p=p)
grid = GridSearchCV(KNeighborsClassifier(), hyperparameters, cv=5,n_jobs=-1 )
grid.fit(X_train, Y_train)
KNN_tuned = grid.best_estimator_
KNN_trained_tuned=KNN_tuned.fit(X_train, Y_train)
survival_predications2=KNN_trained_tuned.predict(X_test)
print("Best parameters: ", grid.best_params_) 


Explanation

#### 6.2.3 Hyperparameter tuning for Random Forest

In [ ]:
param_grid = {'bootstrap': [True, False],
 'max_depth': [10, 20, 40, 60],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [  200, 1000,1600 ,1800, 2000]}

#randomzed search to decrease time frame
grid = RandomizedSearchCV(RandomForestClassifier(), param_grid, refit = True, n_iter = 20, cv = 5, verbose=3, n_jobs =-1)
grid.fit(X_train_RF, Y_train)
RF_tuned = grid.best_estimator_
RF_trained_tuned = RF_tuned.fit(X_train_RF, Y_train) #using the feature tuning sets
print("Best parameters: ", grid.best_params_) 

The Random Forest algorithm is considered a good classifying model. It is a supervised machine learning algorithm. As it was one of the selected algorithms that produced one of the highest cross-validation accuracy scores and one of the lowest cross-validation loss scores, it was chosen to be tuned and retrained. The random forest machine learning algorithm takes the output of multiple decision trees and ‘democratically’ chooses the final outcome. 

Decision trees themselves essentially ask true or false questions that lead to a final answer. In other words, each question (for example, ‘male’ or ‘female’?) is a node, and the outcome is a leaf node. The idea is to have results of subgroups that are similar to one another but are different from the other groups. The random forest works well because while some decision trees may be wrong, many others will be correct. Therefore, as a group, the decision trees can move in the right direction. By using a concept known as ‘bagging,’ where each individual decision tree can randomly sample from the training dataset, with replacement, random forest will have different trees and therefore a low correlation between them. The uncorrelated errors between the different trees means that they ‘protect each other’ from their individual errors. This generally results in a precise classifying algorithm overall. 

Using the RandomizedSearchCV method for time-efficiency, a grid of hyperparameters can be randomly sampled, with K-Fold cross-validation being performed with each combination of parameter values.  The hyperparameters that were searched through include: number of trees in the forest; number of data points per node before it must be split; minimum number of data points allowed in a leaf node; the maximum number of levels of each tree; and with or without replacement. Thereafter, the model is then retrained using the new-found ‘best’ hyperparameters

(Ben)

#### 6.2.2.4 Hyperparameter tuning for LogReg

In [ ]:
param_grid = {'C': np.arange(1e-05, 3, 0.1, 1e-4, 1e-2)}
scoring = {'Accuracy': 'accuracy',
           'AUC': 'roc_auc', 'Log_loss': 'neg_log_loss'}

grid = GridSearchCV(LogisticRegression(), return_train_score=True,
                  param_grid=param_grid, scoring=scoring, cv=10, refit='Accuracy')
grid.fit(X_train_LR, Y_train)
LR_tuned = grid.best_estimator_
LR_trained_tuned=LR_tuned.fit(X_train_LR, Y_train)
gia_output_3=LR_trained_tuned.predict(X_test_LR)
print("Best parameters: ", grid.best_params_) 

Explanation

## Results after tuning:

The results after tuning 4 of the models, as well as the original cross-validation scores for the models that were utilised but not tuned. 

In [ ]:
cross_val_results = []
cross_val_losses = []
# Train using each model, finding accuracy and loss. :
cross_val_results.append(cross_val_score(RF_tuned, X_train_RF, Y_train, scoring = 'accuracy', cv = 5))
cross_val_losses.append(cross_val_score(RF_tuned, X_train_RF, Y_train, scoring = 'neg_log_loss', cv = 5))
cross_val_results.append(cross_val_score(KNN_tuned, X_train, Y_train, scoring = 'accuracy', cv = 5))
cross_val_losses.append(cross_val_score(KNN_tuned, X_train, Y_train, scoring = 'neg_log_loss', cv = 5))
cross_val_results.append(cross_val_score(LR_tuned, X_train_LR, Y_train, scoring = 'accuracy', cv = 5))
cross_val_losses.append(cross_val_score(LR_tuned, X_train_LR, Y_train, scoring = 'neg_log_loss', cv = 5))
cross_val_results.append(cross_val_score(MLP_tuned, X_train, Y_train, scoring = 'accuracy', cv = 5))
cross_val_losses.append(cross_val_score(MLP_tuned, X_train, Y_train, scoring = 'neg_log_loss', cv = 5))

remaining_classifiers = [  
                GaussianNB(),
                SVC(),  
                SGDClassifier(), 
                LinearSVC(),
               ]


for classifier in remaining_classifiers:
    cross_val_results.append(cross_val_score(classifier, X_train, Y_train, scoring = 'accuracy', cv = 5))
    cross_val_losses.append(cross_val_score(classifier, X_train, Y_train, scoring = 'neg_log_loss', cv = 5))

In [ ]:
# Mean and standard deviation of cross validation results for each classifier  

cross_val_mean = []
cross_val_std = []
cross_val_loss_mean =[]
cross_val_loss_std = []

for cross_val_result in cross_val_results:
    cross_val_mean.append(cross_val_result.mean())
    cross_val_std.append(cross_val_result.std())

for cross_val_loss in cross_val_losses:
    cross_val_loss_mean.append(cross_val_loss.mean())
    cross_val_loss_std.append(cross_val_loss.std())    

    
    
#putting our cross val mean and standard deviation for each algorithm into a data frame
cross_val_res = pd.DataFrame({'Cross Validation Mean': cross_val_mean, 'Cross Validation Std': cross_val_std, 'Algorithm': [ 'Random Forest', 'K Nearest Neighbours','Logistic Regression','Multilayer perceptron','Naive Bayes', 'Support Vector Machines', 'Stochastic Gradient Decent', 'Linear SVC']})
cross_val_los = pd.DataFrame({'Cross Validation Mean': cross_val_loss_mean, 'Cross Validation Std': cross_val_loss_std, 'Algorithm':  [ 'Random Forest', 'K Nearest Neighbours','Logistic Regression','Multilayer perceptron','Naive Bayes', 'Support Vector Machines', 'Stochastic Gradient Decent', 'Linear SVC']})
print('Table Showing Cross Validation Mean Accuracy')
print(cross_val_res.sort_values(by = 'Cross Validation Mean', ascending = False, ignore_index = True))
print('\nTable Showing Cross Validation Mean Loss')
print(cross_val_los.sort_values(by = 'Cross Validation Mean', ascending = True, ignore_index = True))

#### Observation 

The tuned algorithms show improvement in accuracy as compared to before being tuned.

### 6.3.4 Ensembles

If cross-val scores were above 80 and the loss was above -2, assign a weighting of 2. Otherwise 1. 

In [ ]:

#train the other models now with no validation set to be used in the ensemble
GNBtrained=GaussianNB().fit(X_train, Y_train)
SVCtrained=SVC().fit(X_train, Y_train)
LSVCtrained=LinearSVC().fit(X_train, Y_train)
SGDtrained=SGDClassifier().fit(X_train, Y_train)

voting_classifier_hard = VotingClassifier(estimators = [('mlp',MLP_trained_tuned),('knn',KNN_trained_tuned),('knn2',KNN_trained_tuned),('lg',LR_trained_tuned),('rf',RF_trained_tuned),('rf2',RF_trained_tuned),('gnb',GNBtrained),('svc',SVCtrained),('svc2',SVCtrained),('lsvc',LSVCtrained),('sgd',SGDtrained)], voting = 'hard') 
voting_classifier_hard_trained=voting_classifier_hard.fit(X_train, Y_train)
survival_predications=voting_classifier_hard_trained.predict(X_test)

print('voting classifier cross validation score :',cross_val_score(voting_classifier_hard,X_train,Y_train,cv=5))
print('voting classifier cross validation score mean :',cross_val_score(voting_classifier_hard,X_train,Y_train,cv=5).mean())
print('voting classifier cross validation score standard deviation :',cross_val_score(voting_classifier_hard,X_train,Y_train,cv=5).std())

# 7. Preparing data for submission

We want our submission dataframe to have 418 rows and 2 columns, PassengerId and Survived. 

In [ ]:
# Create submission dataframe

submit = pd.DataFrame({'PassengerId': test['PassengerId'], 'Survived': survival_predications})
submit.head()

Our dataframe is ready for submission!

In [ ]:
# Create and save csv file 
submit.to_csv("submission.csv", index = False)

# 8. Possible extensions to improve model accuracy

1. Analyse ticket and cabin features
    - Do these features help predict passenger survival?
    - If yes, consider including them in the training set instead of dropping
2. Come up with alternative features in feature engineering
    - Is there any other features you can potentially create from existing features in the dataset
3. Remove features that are less important
    - Does removing features help reduce overfitting in the model?
4. Ensemble modelling
    - This is a more advanced technique whereby you combine prediction results from multiple machine learning models

# 9. Conclusion



## References
https://github.com/chongjason914/kaggle-titanic 

https://www.kaggle.com/code/ldfreeman3/a-data-science-framework-to-achieve-99-accuracy

https://github.com/murilogustineli/Titanic-Classification

https://www.kaggle.com/code/kenjee/titanic-project-example/notebook

